In [ ]:
!pip install s3prl
!pip install librosa
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.9/648.9 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for s3prl: filename=s3prl-0.4.18-py3-none-any.whl size=879859 sha256=b5fa182f89b18f5f143a8402ea35d420ecbb6a25e94985ad11d2808b396a0efb
  Stored in directory: /root/.cache/pip/wheels/91/f6/5a/c06ca16996164f70e8198c7c67c9cdf278572a5a9324c65172
Successfully built s3prl


In [ ]:
import s3prl.hub as hub
import librosa
import torch
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

model_3 = getattr(hub, 'wav2vec2')()  # build the Wav2Vec 2.0 model with pre-trained weights

/usr/local/lib/python3.12/dist-packages/s3prl/upstream/byol_s/byol_a/common.py:20: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  torchaudio.set_audio_backend("sox_io")
/usr/local/lib/python3.12/dist-packages/torchaudio/_internal/module_utils.py:71: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  return func(*args, **kwargs)


Mounted at /content/gdrive


Downloading: https://huggingface.co/s3prl/converted_ckpts/resolve/main/wav2vec_small.pt
Destination: /root/.cache/s3prl/download/aa064e275fe0123a0e1b515f2341bbe4408368510d91d0a6816f2822a6e5acdd.wav2vec_small.pt
100%|██████████| 181M/181M [00:00<00:00, 223MB/s]
/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [ ]:
# y, sr = librosa.load('/content/drive/MyDrive/Research_teaser/q4_audio.wav')
# y_16k = librosa.resample(y, orig_sr=sr, target_sr=16000)
# y_16k_tensor = torch.from_numpy(np.array([y_16k]))
# with torch.no_grad():
#     reps = model_3(y_16k_tensor)["last_hidden_state"]

In [ ]:
import os
import librosa
import numpy as np
import torch

root = "/content/gdrive/MyDrive/unzipped"

all_embeddings = {}

for state in os.listdir(root):
    state_path = os.path.join(root, state)

    if not os.path.isdir(state_path):
        continue

    print(f"\nProcessing state folder: {state}")

    for file in os.listdir(state_path):

        if not file.endswith(".wav"):
            continue

        file_path = os.path.join(state_path, file)
        print("→", file_path)

        # Load audio
        y, sr = librosa.load(file_path, sr=None)

        # Resample to 16k
        y_16k = librosa.resample(y, orig_sr=sr, target_sr=16000)

        # Convert to tensor
        y_16k_tensor = torch.from_numpy(np.array([y_16k])).float()

        # Extract representation
        with torch.no_grad():
            reps = model_3(y_16k_tensor)["last_hidden_state"]

        # Store the result
        all_embeddings[file_path] = reps



Streaming output truncated to the last 5000 lines.
→ /content/gdrive/MyDrive/unzipped/kerala/Kerala_speaker_04_List40_Splitted_4.wav
→ /content/gdrive/MyDrive/unzipped/kerala/Kerala_speaker_04_List41_Splitted_8.wav
→ /content/gdrive/MyDrive/unzipped/kerala/Kerala_speaker_04_List45_Splitted_4.wav
→ /content/gdrive/MyDrive/unzipped/kerala/Kerala_speaker_04_List39_Splitted_2.wav
→ /content/gdrive/MyDrive/unzipped/kerala/Kerala_speaker_04_List47_Splitted_7.wav
→ /content/gdrive/MyDrive/unzipped/kerala/Kerala_speaker_04_List47_Splitted_10.wav
→ /content/gdrive/MyDrive/unzipped/kerala/Kerala_speaker_04_List42_Splitted_8.wav
→ /content/gdrive/MyDrive/unzipped/kerala/Kerala_speaker_04_List39_Splitted_1.wav
→ /content/gdrive/MyDrive/unzipped/kerala/Kerala_speaker_04_List38_Splitted_6.wav
→ /content/gdrive/MyDrive/unzipped/kerala/Kerala_speaker_04_List47_Splitted_8.wav
→ /content/gdrive/MyDrive/unzipped/kerala/Kerala_speaker_04_List48_Splitted_2.wav
→ /content/gdrive/MyDrive/unzipped/kerala/Kera

# *real deal from here*

In [ ]:
root = "/content/drive/MyDrive/unzipped"          # your dataset
save_dir = "/content/gdrive/MyDrive/hubert_layers" # where to save outputs

os.makedirs(save_dir, exist_ok=True)

print("Saving directory:", save_dir)

Saving directory: /content/gdrive/MyDrive/hubert_layers


In [ ]:
def extract_hubert_layers(filepath, savepath):
    # Load & resample
    y, sr = librosa.load(filepath, sr=None)
    y_16k = librosa.resample(y, orig_sr=sr, target_sr=16000)

    # Convert to tensor
    inp = torch.from_numpy(np.array([y_16k])).float()

    # Forward pass
    with torch.no_grad():
        out = hubert(inp)

    hidden_states = out.hidden_states  # tuple of 13 layers

    # Mean-pool each layer
    layer_dict = {}
    for i, layer in enumerate(hidden_states):
        pooled = layer.mean(dim=1).squeeze().cpu()
        layer_dict[f"layer_{i}"] = pooled

    # Save to disk
    torch.save(layer_dict, savepath)


In [ ]:
for state in os.listdir(root):

    state_path = os.path.join(root, state)

    if not os.path.isdir(state_path):
        continue

    print(f"\n📌 Processing state: {state}")

    for file in os.listdir(state_path):

        if not file.endswith(".wav"):
            continue

        file_path = os.path.join(state_path, file)
        save_path = os.path.join(save_dir, file.replace(".wav", ".pt"))

        # Skip if already extracted
        if os.path.exists(save_path):
            print("✔ Exists:", save_path)
            continue

        print("→ Extracting:", file_path)
        extract_hubert_layers(file_path, save_path)
        print("💾 Saved:", save_path)

print("\n🎉 Done! All HuBERT layers extracted.")


Streaming output truncated to the last 5000 lines.
→ Extracting: /content/drive/MyDrive/unzipped/karnataka/Karnataka_speaker_03_1 (91).wav
💾 Saved: /content/gdrive/MyDrive/hubert_layers/Karnataka_speaker_03_1 (91).pt
→ Extracting: /content/drive/MyDrive/unzipped/karnataka/Karnataka_speaker_03_1 (79).wav
💾 Saved: /content/gdrive/MyDrive/hubert_layers/Karnataka_speaker_03_1 (79).pt
→ Extracting: /content/drive/MyDrive/unzipped/karnataka/Karnataka_speaker_03_1 (721).wav
💾 Saved: /content/gdrive/MyDrive/hubert_layers/Karnataka_speaker_03_1 (721).pt
→ Extracting: /content/drive/MyDrive/unzipped/karnataka/Karnataka_speaker_03_1 (75).wav
💾 Saved: /content/gdrive/MyDrive/hubert_layers/Karnataka_speaker_03_1 (75).pt
→ Extracting: /content/drive/MyDrive/unzipped/karnataka/Karnataka_speaker_03_1 (84).wav
💾 Saved: /content/gdrive/MyDrive/hubert_layers/Karnataka_speaker_03_1 (84).pt
→ Extracting: /content/drive/MyDrive/unzipped/karnataka/Karnataka_speaker_03_1 (730).wav
💾 Saved: /content/gdrive/MyD

In [ ]:
!zip -r hubert_layers.zip /content/gdrive/MyDrive/hubert_layers


Streaming output truncated to the last 5000 lines.
  adding: content/gdrive/MyDrive/hubert_layers/Karnataka_speaker_01_List11_4.pt (deflated 14%)
  adding: content/gdrive/MyDrive/hubert_layers/Kerala_speaker_03_malayalam_s01_526.pt (deflated 15%)
  adding: content/gdrive/MyDrive/hubert_layers/Karnataka_speaker_03_1 (449).pt (deflated 14%)
  adding: content/gdrive/MyDrive/hubert_layers/Andhra_speaker (439).pt (deflated 14%)
  adding: content/gdrive/MyDrive/hubert_layers/Kerala_speaker_04_List28_Splitted_8.pt (deflated 14%)
  adding: content/gdrive/MyDrive/hubert_layers/Gujrat_speaker_02_5.pt (deflated 14%)
  adding: content/gdrive/MyDrive/hubert_layers/Karnataka_speaker_02_1 (75).pt (deflated 14%)
  adding: content/gdrive/MyDrive/hubert_layers/Karnataka_speaker_02_1 (729).pt (deflated 14%)
  adding: content/gdrive/MyDrive/hubert_layers/Andhra_speaker (1049).pt (deflated 14%)
  adding: content/gdrive/MyDrive/hubert_layers/Andhra_speaker (1559).pt (deflated 14%)
  adding: content/gdrive/M

In [ ]:
from google.colab import files
files.download("hubert_layers.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

run from here


In [ ]:
!pip install transformers

import os
import torch
import librosa
import numpy as np

from transformers import Wav2Vec2FeatureExtractor, HubertModel
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


In [ ]:
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(
    "facebook/hubert-base-ls960"
)

hubert = HubertModel.from_pretrained(
    "facebook/hubert-base-ls960",
    output_hidden_states=True
)

hubert.eval()
print("HuBERT Loaded ✔")


HuBERT Loaded ✔


In [ ]:
import os
import torch
import numpy as np

layer_dir = "/content/drive/MyDrive/hubert_layers"

files = [f for f in os.listdir(layer_dir) if f.endswith(".pt")]
print("Total files:", len(files))

X_layers = {i: [] for i in range(13)}
labels = []
filenames = []

for f in files:
    data = torch.load(os.path.join(layer_dir, f))

    # extract 13 layer vectors
    for i in range(13):
        X_layers[i].append(data[f"layer_{i}"].numpy())

    # label is first token before "_" or space
    label = f.split("_")[0].split(" ")[0]
    labels.append(label)
    filenames.append(f)

print("Loaded embeddings.")


Total files: 8116
Loaded embeddings.


In [ ]:
from collections import Counter

label_counts = Counter(labels)
print("Label counts:", label_counts)

low_labels = [lbl for lbl, cnt in label_counts.items() if cnt < 5]
print("\n⚠️ Low-sample labels (<5):", low_labels)


Label counts: Counter({'Tamil': 1840, 'Andhra': 1794, 'Karnataka': 1686, 'Kerala': 1471, 'Jharkhand': 827, 'Gujrat': 298, 'WhatsApp': 200})

⚠️ Low-sample labels (<5): []


In [ ]:
X_layers_clean = {i: [] for i in range(13)}
y_clean = []
filenames_clean = []

for idx, lbl in enumerate(labels):
    if lbl not in low_labels:     # keep only good labels
        for i in range(13):
            X_layers_clean[i].append(X_layers[i][idx])

        y_clean.append(lbl)
        filenames_clean.append(filenames[idx])

print(f"Kept {len(y_clean)} samples. Removed {len(labels) - len(y_clean)} bad ones.")


Kept 8116 samples. Removed 0 bad ones.


In [ ]:
for i in range(13):
    X_layers_clean[i] = np.array(X_layers_clean[i])

y_clean = np.array(y_clean)

print("Shapes:")
print("y_clean:", y_clean.shape)
print("Example X:", X_layers_clean[0].shape)


Shapes:
y_clean: (8116,)
Example X: (8116, 768)


In [ ]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
y_enc = enc.fit_transform(y_clean)

print("Classes:", enc.classes_)


Classes: ['Andhra' 'Gujrat' 'Jharkhand' 'Karnataka' 'Kerala' 'Tamil' 'WhatsApp']


In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

idx_train, idx_test = train_test_split(
    np.arange(len(y_enc)),
    test_size=0.25,
    random_state=42,
    stratify=y_enc
)

print("Split OK:")
print("Train:", len(idx_train))
print("Test:", len(idx_test))


Split OK:
Train: 6087
Test: 2029


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

layer_accuracies = {}

for layer in range(13):
    print(f"\nTraining SVM for Layer {layer}...")

    X_train = X_layers_clean[layer][idx_train]
    X_test  = X_layers_clean[layer][idx_test]

    clf = SVC(kernel="rbf", C=10, gamma="scale")
    clf.fit(X_train, y_enc[idx_train])

    pred = clf.predict(X_test)
    acc = accuracy_score(y_enc[idx_test], pred)

    layer_accuracies[layer] = acc
    print(f"Layer {layer}: {acc:.4f}")



Training SVM for Layer 0...
Layer 0: 0.9985

Training SVM for Layer 1...
Layer 1: 0.9985

Training SVM for Layer 2...
Layer 2: 0.9990

Training SVM for Layer 3...
Layer 3: 0.9985

Training SVM for Layer 4...
Layer 4: 0.9985

Training SVM for Layer 5...
Layer 5: 0.9985

Training SVM for Layer 6...
Layer 6: 0.9985

Training SVM for Layer 7...
Layer 7: 0.9980

Training SVM for Layer 8...
Layer 8: 0.9975

Training SVM for Layer 9...
Layer 9: 0.9961

Training SVM for Layer 10...
Layer 10: 0.9975

Training SVM for Layer 11...
Layer 11: 0.9966

Training SVM for Layer 12...
Layer 12: 0.9966


In [ ]:
import torch

data = torch.load("/content/drive/MyDrive/hubert_layers/Andhra_speaker (1005).pt")
print(type(data))
print(data.keys())

for k, v in data.items():
    print(k, v.shape)

print(data["layer_0"].dtype)


<class 'dict'>
dict_keys(['layer_0', 'layer_1', 'layer_2', 'layer_3', 'layer_4', 'layer_5', 'layer_6', 'layer_7', 'layer_8', 'layer_9', 'layer_10', 'layer_11', 'layer_12'])
layer_0 torch.Size([768])
layer_1 torch.Size([768])
layer_2 torch.Size([768])
layer_3 torch.Size([768])
layer_4 torch.Size([768])
layer_5 torch.Size([768])
layer_6 torch.Size([768])
layer_7 torch.Size([768])
layer_8 torch.Size([768])
layer_9 torch.Size([768])
layer_10 torch.Size([768])
layer_11 torch.Size([768])
layer_12 torch.Size([768])
torch.float32


# **Sequence level extraction for CNN and BLiSTM and Transformers**

In [ ]:
!pip install transformers torch librosa soundfile tqdm


In [ ]:
import torch
from transformers import HubertModel
import librosa
import soundfile as sf
import os
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using:", device)

hubert = HubertModel.from_pretrained("facebook/hubert-base-ls960").to(device)
hubert.eval()


Using: cpu


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

HubertModel(
  (feature_extractor): HubertFeatureEncoder(
    (conv_layers): ModuleList(
      (0): HubertGroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1-4): 4 x HubertNoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (5-6): 2 x HubertNoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): HubertFeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): HubertEncoder(
    (pos_conv_embed): HubertPositionalConvEmbedding(
      (conv): Para

In [ ]:
def extract_hubert_sequence(filepath, savepath):
    try:
        # Load audio
        y, sr = librosa.load(filepath, sr=None)

        # Resample to 16k (HuBERT requirement)
        if sr != 16000:
            y = librosa.resample(y, orig_sr=sr, target_sr=16000)

        # Convert to tensor: (1, T)
        inp = torch.tensor(y, dtype=torch.float32).unsqueeze(0).to(device)

        # Forward pass
        with torch.no_grad():
            out = hubert(inp)

        # Last hidden state (sequence-level)
        seq = out.last_hidden_state.squeeze(0).cpu()   # shape: (T, 768)

        # Save safely
        torch.save(seq, savepath)

    except Exception as e:
        print(f"❌ Error in file {filepath}: {e}")


In [ ]:
root = "/content/drive/MyDrive/unzipped"   # your audio files
save_dir = "/content/drive/MyDrive/hubert_sequence"   # where .pt files go

os.makedirs(save_dir, exist_ok=True)

for state in os.listdir(root):
    state_path = os.path.join(root, state)

    if not os.path.isdir(state_path):
        continue

    print(f"\n Processing state: {state}")

    for file in tqdm(os.listdir(state_path)):
        if not file.lower().endswith(".wav"):
            continue

        file_path = os.path.join(state_path, file)
        save_path = os.path.join(save_dir, f"{file.replace('.wav', '')}.pt")

        # Skip if exists
        if os.path.exists(save_path):
            continue

        extract_hubert_sequence(file_path, save_path)

print("\n Done! All HuBERT sequence embeddings extracted.")



 Processing state: jharkhand


100%|██████████| 827/827 [26:08<00:00,  1.90s/it]



 Processing state: tamil


100%|██████████| 1840/1840 [1:01:48<00:00,  2.02s/it]



 Processing state: kerala


...


 Processing state: karnataka


100%|██████████| 1686/1686 [1:23:53<00:00,  2.99s/it]



 Processing state: andhra_pradesh


100%|██████████| 1794/1794 [1:05:48<00:00,  2.20s/it]


 Done! All HuBERT sequence embeddings extracted.


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import glob, os

class SeqDataset(Dataset):
    def __init__(self, root):
        self.files = []
        self.labels = []
        for f in glob.glob(os.path.join(root, "*.pt")):
            name = os.path.basename(f).lower()
            lbl = None
            for s in label_to_idx:
                if s in name:
                    lbl = label_to_idx[s]
                    break
            if lbl is None:
                continue
            self.files.append(f)
            self.labels.append(lbl)

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        x = torch.load(self.files[idx])
        y = torch.tensor(self.labels[idx])
        return x, y


def collate_fn(batch):
    xs, ys = zip(*batch)
    max_len = 300
    padded = []

    for x in xs:
        if x.shape[0] > max_len:
            x = x[:max_len]
        else:
            pad = max_len - x.shape[0]
            x = torch.cat([x, torch.zeros(pad, x.shape[1])], 0)
        padded.append(x)

    return torch.stack(padded), torch.tensor(ys)


In [ ]:
label_to_idx = {
    "andhra":0,
    "gujrat":1,
    "jharkhand":2,
    "karnataka":3,
    "kerala":4,
    "tamil":5
}


In [ ]:
#CNN
class CNN1D(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(768, 256, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(256, 128, 5, padding=2),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)
        )
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        x = x.transpose(1, 2)   # (B, 768, T)
        x = self.conv(x)
        x = x.squeeze(-1)
        return self.fc(x)


In [ ]:
#BiLSTM

class BiLSTM(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.lstm = nn.LSTM(768, 256, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        return self.fc(out)


In [ ]:
#Transformer model
class TransformerModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=768,
            nhead=8,
            batch_first=True,
            dim_feedforward=1024
        )
        self.tf = nn.TransformerEncoder(encoder_layer, num_layers=2)
        self.fc = nn.Linear(768, num_classes)

    def forward(self, x):
        out = self.tf(x)
        out = out.mean(dim=1)   # average pooling
        return self.fc(out)


In [ ]:
#TRAIN

def train(model, loader, epochs=5):
    model = model.to(device)             # <<< MOVE MODEL TO GPU
    opt = torch.optim.Adam(model.parameters(), lr=1e-4)
    loss_fn = nn.CrossEntropyLoss()

    for e in range(epochs):
        total = 0
        correct = 0

        for x, y in loader:
            x = x.to(device)             # <<< MOVE INPUT TO GPU
            y = y.to(device)             # <<< MOVE LABELS TO GPU

            opt.zero_grad()
            out = model(x)
            loss = loss_fn(out, y)
            loss.backward()
            opt.step()

            pred = out.argmax(1)
            correct += (pred == y).sum().item()
            total += y.size(0)

        print(f"Epoch {e+1}: Acc = {correct/total:.4f}")

In [ ]:
torch.cuda.is_available()

True

In [ ]:
#Run all 3

train_data = SeqDataset("/content/drive/MyDrive/hubert_sequence")
loader = DataLoader(train_data, batch_size=8, shuffle=True, collate_fn=collate_fn)

cnn = CNN1D(num_classes=6)
lstm = BiLSTM(num_classes=6)
tfm = TransformerModel(num_classes=6)

# print("Training CNN…")
# train(cnn, loader)

# print("Training BiLSTM…")
# train(lstm, loader)

# print("Training Transformer…")
# train(tfm, loader)



In [ ]:
train(tfm, loader)


Epoch 1: Acc = 0.9368
Epoch 2: Acc = 0.9888
Epoch 3: Acc = 0.9928
Epoch 4: Acc = 0.9923
Epoch 5: Acc = 0.9967


In [ ]:
train(cnn, loader)


Epoch 1: Acc = 0.8545
Epoch 2: Acc = 0.9848
Epoch 3: Acc = 0.9914
Epoch 4: Acc = 0.9944
Epoch 5: Acc = 0.9966


In [ ]:

train(lstm, loader)


Epoch 1: Acc = 0.2644
Epoch 2: Acc = 0.2748
Epoch 3: Acc = 0.3137
Epoch 4: Acc = 0.3099
Epoch 5: Acc = 0.3747


In [ ]:
os.makedirs("/content/drive/MyDrive/models", exist_ok=True)


In [ ]:
torch.save(cnn.state_dict(), "/content/drive/MyDrive/models/cnn_seq.pt")
torch.save(lstm.state_dict(), "/content/drive/MyDrive/models/lstm_seq.pt")
torch.save(tfm.state_dict(), "/content/drive/MyDrive/models/transformer_seq.pt")


# **Word vs Sentence Analysis**

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import glob, os
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

label_to_idx = {
    "andhra":0,
    "gujrat":1,
    "jharkhand":2,
    "karnataka":3,
    "kerala":4,
    "tamil":5
}

class SeqDataset(Dataset):
    def __init__(self, root):
        self.files = []
        self.labels = []
        for f in glob.glob(os.path.join(root, "*.pt")):
            name = os.path.basename(f).lower()
            lbl = None
            for s in label_to_idx:
                if s in name:
                    lbl = label_to_idx[s]
                    break
            if lbl is None:
                continue
            self.files.append(f)
            self.labels.append(lbl)

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        x = torch.load(self.files[idx])   # (T,768)
        y = torch.tensor(self.labels[idx])
        return x, y

def collate_fn(batch, max_len=300):
    xs, ys = zip(*batch)
    padded = []
    for x in xs:
        if x.shape[0] > max_len:
            x = x[:max_len]
        else:
            pad = max_len - x.shape[0]
            x = torch.cat([x, torch.zeros(pad, x.shape[1])], dim=0)
        padded.append(x)
    return torch.stack(padded), torch.tensor(ys)


In [2]:
full_ds = SeqDataset("/content/drive/MyDrive/hubert_sequence")

lengths = []
for f in full_ds.files:
    x = torch.load(f)
    lengths.append(x.shape[0])

lengths = np.array(lengths)
print("Min T:", lengths.min(), "Max T:", lengths.max())
print("Percentiles 25/50/75:", np.percentile(lengths, [25, 50, 75]))


Min T: 11 Max T: 1129
Percentiles 25/50/75: [163. 229. 249.]


In [3]:
word_max = 160   # <= this = word-level
sent_min = 260   # >= this = sentence-level

word_indices = [i for i, T in enumerate(lengths) if T <= word_max]
sent_indices = [i for i, T in enumerate(lengths) if T >= sent_min]

print("Word-like samples:", len(word_indices))
print("Sentence-like samples:", len(sent_indices))


Word-like samples: 1869
Sentence-like samples: 1023


In [4]:
from torch.utils.data import Subset

word_ds = Subset(full_ds, word_indices)
sent_ds = Subset(full_ds, sent_indices)

device = "cuda" if torch.cuda.is_available() else "cpu"

word_loader = DataLoader(
    word_ds,
    batch_size=64,
    shuffle=False,
    num_workers=2,
    pin_memory=True,
    collate_fn=collate_fn
)

sent_loader = DataLoader(
    sent_ds,
    batch_size=64,
    shuffle=False,
    num_workers=2,
    pin_memory=True,
    collate_fn=collate_fn
)


In [5]:
def evaluate_torch_model_with_conf(model, loader, device="cuda"):
    model.eval()
    all_preds, all_trues = [], []
    confidences = []

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            out = model(x)
            probs = torch.softmax(out, dim=1)
            preds = probs.argmax(1).cpu().numpy()

            all_preds.extend(preds)
            all_trues.extend(y.numpy())
            confidences.extend(probs.max(dim=1).values.cpu().numpy())

    acc = accuracy_score(all_trues, all_preds)
    f1_macro = f1_score(all_trues, all_preds, average="macro")
    f1_weighted = f1_score(all_trues, all_preds, average="weighted")
    cm = confusion_matrix(all_trues, all_preds)
    rpt = classification_report(all_trues, all_preds)

    avg_conf = float(np.mean(confidences))

    return acc, f1_macro, f1_weighted, avg_conf, cm, rpt


In [7]:
import torch
import torch.nn as nn

class CNN1D_BN(nn.Module):
    def __init__(self, num_classes=6):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(768, 256, 5, padding=2),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Conv1d(256, 128, 5, padding=2),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)
        )
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.conv(x).squeeze(-1)
        return self.fc(x)


In [8]:
cnn_bn_model = torch.load(
    "/content/drive/MyDrive/models/cnn_bn_fullmodel.pt",
    map_location=device,
    weights_only=False
).to(device)
cnn_bn_model.eval()


CNN1D_BN(
  (conv): Sequential(
    (0): Conv1d(768, 256, kernel_size=(5,), stride=(1,), padding=(2,))
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv1d(256, 128, kernel_size=(5,), stride=(1,), padding=(2,))
    (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): AdaptiveAvgPool1d(output_size=1)
  )
  (fc): Linear(in_features=128, out_features=6, bias=True)
)

In [10]:
print("=== CNN-BN on WORD-like samples ===")
acc, f1m, f1w, avg_conf, cm, rpt = evaluate_torch_model_with_conf(cnn_bn_model, word_loader, device)
print("Accuracy:", acc)
print("Macro F1:", f1m)
print("Weighted F1:", f1w)
print("Avg confidence:", avg_conf)
print("Confusion Matrix:\n", cm)
print("Report:\n", rpt)

print("\n=== CNN-BN on SENTENCE-like samples ===")
acc, f1m, f1w, avg_conf, cm, rpt = evaluate_torch_model_with_conf(cnn_bn_model, sent_loader, device)
print("Accuracy:", acc)
print("Macro F1:", f1m)
print("Weighted F1:", f1w)
print("Avg confidence:", avg_conf)
print("Confusion Matrix:\n", cm)
print("Report:\n", rpt)


=== CNN-BN on WORD-like samples ===
Accuracy: 0.9962546816479401
Macro F1: 0.996259458079601
Weighted F1: 0.9962495886603961
Avg confidence: 0.8921356797218323
Confusion Matrix:
 [[847   0   0   0   0   0]
 [  1  62   0   0   0   0]
 [  0   0 162   0   0   0]
 [  0   0   0  13   0   0]
 [  1   0   0   0 209   2]
 [  3   0   0   0   0 569]]
Report:
               precision    recall  f1-score   support

           0       0.99      1.00      1.00       847
           1       1.00      0.98      0.99        63
           2       1.00      1.00      1.00       162
           3       1.00      1.00      1.00        13
           4       1.00      0.99      0.99       212
           5       1.00      0.99      1.00       572

    accuracy                           1.00      1869
   macro avg       1.00      0.99      1.00      1869
weighted avg       1.00      1.00      1.00      1869


=== CNN-BN on SENTENCE-like samples ===


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Accuracy: 1.0
Macro F1: 1.0
Weighted F1: 1.0
Avg confidence: 0.9576871395111084
Confusion Matrix:
 [[199   0   0   0   0   0]
 [  0   5   0   0   0   0]
 [  0   0 276   0   0   0]
 [  0   0   0 253   0   0]
 [  0   0   0   0 171   0]
 [  0   0   0   0   0 119]]
Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       199
           1       1.00      1.00      1.00         5
           2       1.00      1.00      1.00       276
           3       1.00      1.00      1.00       253
           4       1.00      1.00      1.00       171
           5       1.00      1.00      1.00       119

    accuracy                           1.00      1023
   macro avg       1.00      1.00      1.00      1023
weighted avg       1.00      1.00      1.00      1023

